https://www.youtube.com/watch?v=KYFfMymTspE

In [10]:
from keras.applications.densenet import DenseNet121, decode_predictions
from keras.models import Sequential,Model 
from keras.layers import Dense,Dropout,Activation,Flatten,GlobalAveragePooling2D
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator 
from keras import backend as K

In [ ]:
keras.applications.densenet.DenseNet121(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)

In [11]:
img_rows,img_cols = 224, 224

densenet = DenseNet121(weights='imagenet',include_top=False,input_shape=(img_rows,img_cols,3))

for layer in Resnet.layers:
    layer.trainable=False

def addTopModeldensenet(bottom_model, num_classes):

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(256,activation='relu')(top_model)
    top_model = Dense(256,activation='relu')(top_model)
    top_model = Dense(128,activation='relu')(top_model)
    top_model = Dense(128,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)

    return top_model

num_classes = 11

FC_Head = addTopModelResnet(densenet,num_classes)

model = Model(inputs = densenet.input, outputs = FC_Head)

print(model.summary())

29089792/29084464 [==============================] - 4s 0us/step
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
___________________________

In [12]:
train_data_dir = 'train'
validation_data_dir = 'test'

train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=30,
                    width_shift_range=0.3,
                    height_shift_range=0.3,
                    horizontal_flip=True,
                    fill_mode='nearest'
                                   )

validation_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32

train_generator = train_datagen.flow_from_directory(
                        train_data_dir,
                        target_size = (img_rows,img_cols),
                        batch_size = batch_size,
                        class_mode = 'categorical'
                        )

validation_generator = validation_datagen.flow_from_directory(
                            validation_data_dir,
                            target_size=(img_rows,img_cols),
                            batch_size=batch_size,
                            class_mode='categorical')

from keras.optimizers import RMSprop,Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

checkpoint = ModelCheckpoint(
                             'finger_sign_densenet121.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(
                          monitor='val_loss',
                          min_delta=0,
                          restore_best_weights=True,
                          patience=10,
                          verbose=1)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=5, 
                                            verbose=1, 
                                            factor=0.2, 
                                            min_lr=0.0001)

callbacks = [earlystop,checkpoint,learning_rate_reduction]

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy']
              )

nb_train_samples = 1200
nb_validation_samples = 300  

epochs = 10
batch_size = 32

history = model.fit_generator(
            train_generator,
            steps_per_epoch=nb_train_samples//batch_size,
            epochs=epochs,
            callbacks=callbacks,
            validation_data=validation_generator,
            validation_steps=nb_validation_samples//batch_size)


Found 5262 images belonging to 11 classes.
Found 5262 images belonging to 11 classes.
Epoch 1/10
37/37 [==============================] - 88s 2s/step - loss: 1.4917 - accuracy: 0.4899 - val_loss: 19.7682 - val_accuracy: 0.1806

Epoch 00001: val_loss improved from inf to 19.76824, saving model to finger_sign_densenet121.h5
Epoch 2/10
37/37 [==============================] - 32s 862ms/step - loss: 0.6594 - accuracy: 0.7762 - val_loss: 112.0996 - val_accuracy: 0.1528

Epoch 00002: val_loss did not improve from 19.76824
Epoch 3/10
37/37 [==============================] - 34s 932ms/step - loss: 0.3622 - accuracy: 0.8851 - val_loss: 105.2004 - val_accuracy: 0.1181

Epoch 00003: val_loss did not improve from 19.76824
Epoch 4/10
37/37 [==============================] - 32s 873ms/step - loss: 0.1169 - accuracy: 0.9628 - val_loss: 20.3906 - val_accuracy: 0.5035

Epoch 00004: val_loss did not improve from 19.76824
Epoch 5/10
37/37 [==============================] - 32s 875ms/step - loss: 0.2833 -